# Fetch Foursquare Venues

## Installation

### Enable extansions
```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

### Install ipyleaflet
```
$ pip3 install ipyleaflet
$ jupyter nbextension enable --py --sys-prefix ipyleaflet
```

### Install foursquare lib
```
$ pip3 install foursquare
```

### Setup foursquare

Set ```client_id``` and ```client_secret``` in Config section

## Config

In [6]:
bb_tl = (59.8946, 29.8735)
bb_br = (59.8657, 29.9534)

In [7]:
client_id = '<CLIENT_ID>'
client_secret = '<CLIENT_SECRET>'
access_token = ''

## Initialization

In [8]:
import foursquare
api = foursquare.Foursquare(client_id=client_id, client_secret=client_secret)

In [9]:
from __future__ import print_function
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [10]:
from random import random


def randrange(start, end):
    return start + random() * (end - start)


def random_coord(bb):
    bb_tl, bb_br = bb

    lat = randrange(bb_br[0], bb_tl[0])
    lon = randrange(bb_br[1], bb_tl[1])
    return lat, lon

In [11]:
import json

def to_geojson(venues):
    j = {
        "type": "FeatureCollection",
        "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::4326" } },
        "features": list(map(to_feature, venues))
    }
    return json.dumps(j, indent=4, ensure_ascii=False)

def to_feature(venue):
    name, lat, lon, checkins, users, cat = venue
    return {
        "type": "Feature",
        "properties": {
            'name': name,
            'checkins': checkins,
            'users': users,
            'category': cat,
        },
        "geometry": {
            "type": "Point",
            "coordinates": [lon, lat]
        }
    }

In [12]:
def api_venues(coord):
    query = {
        'll': '%s,%s' % coord,
        'limit': 50,
    }

    res = api.venues.search(params=query)
    venues = res['venues']
    return venues

In [13]:
def map_number(value, s1, s2, d1, d2):
    r = float(value - s1) / (s2 - s1)
    return d1 + (d2 - d1) * r

## Load Venues

In [14]:
bb = [bb_tl, bb_br]
random_coords = [random_coord(bb) for i in range(10)]
api_reqs = [api_venues(coord) for coord in random_coords]

In [15]:
def cats(cs):
    return list(map(lambda c: c['name'], cs))

from itertools import chain
venues = list(chain.from_iterable(api_reqs))
venues = dict(map(lambda v: (v['id'], v), venues))
venues = list(venues.values())
venues = list(map(
    lambda v: (
        v['name'],
        v['location']['lat'],
        v['location']['lng'],
        v['stats']['checkinsCount'],
        v['stats']['usersCount'],
        cats(v['categories']),
    ),
    venues
))
venues[0]

('пруд с осетрами', 59.8890191187011, 29.895963610318624, 21, 21, ['Beach'])

## Save to Checkins.geojson

In [16]:
with open('Checkins.geojson', 'w') as f:
    f.write(to_geojson(venues))

## Display map

In [17]:
center = (59.8853, 29.8975)
m = Map(center=center, zoom=15)

for name, lat, lon, checkins, users, cat in venues:
    radius = int(map_number(checkins, 1000, 100000, 10, 100))
    circle = Circle(location=[lat, lon], radius=radius, opacity=0, fill_opacity=1.0)
    m.add_layer(circle)
        
Q = Marker(location=center)
m.add_layer(Q)
m

Map(center=[59.8853, 29.8975], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Circle(fill_opacity=1.0, location=[59.8890191187011, 29.895963610318624], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.88681797288767, 29.893109650379657], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.88655380427957, 29.896159373721474], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.889356071144505, 29.89805602329371], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.887055, 29.891386], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.886669443330376, 29.89215809577251], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.888673727499196, 29.897029982277616], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=10), Circle(fill_opacity=1.0, location=[59.886773, 29.896958], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.887817334829826, 29.897596836090088], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.88608962976415, 29.89297022015517], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.886404354974516, 29.893221842227486], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.887273, 29.892153], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.886539459228516, 29.89740753173828], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.88648532027226, 29.892680752208843], opacity=0.0, options=['class_name', 'clickable', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'stroke', 'weight'], radius=9), Circle(fill_opacity=1.0, location=[59.886051521173464, 29.893454368433666], opacity=0.0, options=['class_name', 'cl